In [2]:
import keras
import time
# from mtcnn.mtcnn import MTCNN
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
from numpy import array
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout

In [3]:
imgs = glob.glob("Data/*.jpg")

In [4]:
faces_imgs = np.load("faces_imgs.npy")
faces_annot = np.load("faces_annot.npy")[0]

In [8]:
get_id_coord(imgs[52])

array([[1.        , 0.1953125 , 0.25036603, 0.38085938, 0.57101023]])

In [ ]:
plot_box(get_id_coord(imgs[36]), imgs[36])

In [6]:
def get_face_id(img):
    faces = []
    for i in faces_imgs:
        if i[1] == img:
            faces.append(i[0])
    return np.array(faces)

In [7]:
def get_id_coord(img):
    W, H = Image.open(img).size
    coord = [] 
    ids = get_face_id(img[5:])
    for i in ids:
        coord.append(faces_annot[int(i)])
    a = np.array(coord, dtype=np.float32).reshape(-1, 4)
    a = a[a[:,0].argsort()]
    a[:, 0] = a[:, 0] /  W 
    a[:, 1] = a[:, 1] /  H 
    a[:, 2] = a[:, 2] /  W 
    a[:, 3] = a[:, 3] /  H 

    a = np.append(arr = np.ones((len(a), 1)), values= a, axis = 1).reshape(-1, 5)
    return a

In [9]:
K.set_learning_phase(1)
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.85)
sess = tf.Session()

X = tf.placeholder(dtype = tf.float32, shape = (1, None, None, 3))
Y = tf.placeholder(dtype = tf.float32, shape = (None, 5))

In [10]:
vgg = VGGFace(include_top=False, input_tensor = X, input_shape=(None, None, 3))
for layer in vgg.layers:
    layer.trainable = False

In [11]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, None, None, 64)    1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, None, None, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, None, None, 64)    0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, None, None, 128)   73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, None, None, 128)   147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, None, None, 128)   0         
__________

In [12]:
p1 = Conv2D(filters=1024, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=11).output)
p1 = BatchNormalization()(p1)
p1 = Activation('relu')(p1)
p21 = Conv2D(filters=1024, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=13).output)
p2 = BatchNormalization()(p21)
p2 = Activation('relu')(p2)

p101 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p102 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p102 = BatchNormalization()(p102)
# p102 = Activation('relu')(p102)
# p102 = Activation('sigmoid')(p102)
p201 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p202 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p202 = BatchNormalization()(p202)
# p202 = Activation('relu')(p202)
# p202 = Activation('sigmoid')(p202)


# DownSample

p1 = MaxPool2D(strides= 2)(p1)
p2 = MaxPool2D(strides= 2)(p2)

p1 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(p1)
p1 = BatchNormalization()(p1)
p1 = Activation('relu')(p1)
p2 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(p2)
p2 = BatchNormalization()(p2)
p2 = Activation('relu')(p2)

p111 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p112 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p112 = BatchNormalization()(p112)
# p112 = Activation('relu')(p112)
# p112 = Activation('sigmoid')(p112)
p211 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p212 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p212 = BatchNormalization()(p212)
# p212 = Activation('relu')(p212)
# p212 = Activation('sigmoid')(p212)

# DownSample

p1 = MaxPool2D(strides= 2)(p1)
p2 = MaxPool2D(strides= 2)(p2)

p1 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(p1)
p1 = BatchNormalization()(p1)
p1 = Activation('relu')(p1)
p2 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(p2)
p2 = BatchNormalization()(p2)
p2 = Activation('relu')(p2)

p121 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p122 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p122 = BatchNormalization()(p122)
# p122 = Activation('relu')(p122)
# p122 = Activation('sigmoid')(p122)
p221 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p222 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p222 = BatchNormalization()(p222)
# p222 = Activation('relu')(p222)
# p222 = Activation('sigmoid')(p222)

# DownSample

p1 = MaxPool2D(strides= 2)(p1)
p2 = MaxPool2D(strides= 2)(p2)

p1 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(p1)
p1 = BatchNormalization()(p1)
p1 = Activation('relu')(p1)
p2 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(p2)
p2 = BatchNormalization()(p2)
p2 = Activation('relu')(p2)

p131 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p132 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p132 = BatchNormalization()(p132)
# p132 = Activation('relu')(p132)
# p132 = Activation('sigmoid')(p132)
p231 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p232 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p232 = BatchNormalization()(p232)
# p232 = Activation('relu')(p232)
# p232 = Activation('sigmoid')(p232)

# Combine All Predictions

p102 = K.concatenate(tensors=[p101, p102], axis = -1)
p202 = K.concatenate(tensors=[p201, p202], axis = -1)

p112 = K.concatenate(tensors=[p111, p112], axis = -1)
p212 = K.concatenate(tensors=[p211, p212], axis = -1)

p122 = K.concatenate(tensors=[p121, p122], axis = -1)
p222 = K.concatenate(tensors=[p221, p222], axis = -1)

p132 = K.concatenate(tensors=[p131, p132], axis = -1)
p232 = K.concatenate(tensors=[p231, p232], axis = -1)

In [13]:
preds = [K.reshape(p102, (-1, 5)),
        K.reshape(p202, (-1, 5)),
        K.reshape(p112, (-1, 5)),
        K.reshape(p212, (-1, 5)),
        K.reshape(p122, (-1, 5)),
        K.reshape(p222, (-1, 5)),
        K.reshape(p132, (-1, 5)),
        K.reshape(p232, (-1, 5))
       ]

In [14]:
preds = K.concatenate(preds, axis = 0)

In [15]:
def NMS(boxes = preds):    
    Z0 = tf.image.non_max_suppression(boxes=K.reshape(boxes[:, 1:], (-1, 4)), scores=K.reshape(boxes[:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    A = K.reshape(tf.gather_nd(boxes, K.reshape(Z0, (-1, 1))), (-1, 5))
    return A

In [16]:
def loss(Y, y, a=0.8):
    if tf.size(Y) != tf.size(y):
        n = tf.truncatediv(tf.size(Y),  K.constant(value = 5, dtype=tf.int32))
        y_ = y[:n]
        
    softMax = K.mean(-(Y[:, 0] * K.log(y_[:, 0]) + (1 - Y[:, 0]) * K.log(1 - y_[:, 0]))) + 0.15 * K.mean(-(tf.zeros_like(y[n:])[:, 0] * K.log(y[n:][:, 0]) + (1 - tf.zeros_like(y[n:])[:, 0]) * K.log(1 - y[n:][:, 0])))
    mse = K.mean(K.square(Y[:, 1:] - y_[:, 1:]))
    return softMax + 1.25 * mse

In [17]:
bbox = tf.abs(NMS())

In [18]:
cost = loss(Y, bbox)

In [19]:
opt = tf.train.AdamOptimizer(learning_rate=0.0000005)
train_ = opt.minimize(cost)

In [20]:
saver = tf.train.Saver()

In [21]:
sess = tf.InteractiveSession()

In [22]:
sess.run(tf.global_variables_initializer())

In [23]:
def train(ID, it, epochs = 10):
    for e in range(epochs):
        loss = 0.0
        for i in range(len(imgs)):
#             print(imgs[i])
            _, L = sess.run([train_, cost], feed_dict={X: np.expand_dims(np.array(Image.open(imgs[i]).convert("RGB").resize((100, 100), Image.ANTIALIAS))/255, axis = 0), Y: get_id_coord(imgs[i]).reshape(-1, 5).astype(float)})       
#             print("DOne")
            loss += L
            if i % 500 == 0:
                print("iter {} loss: {}".format(i, L))
                saver.save(sess, save_path="checkpoints/epoch_{}_iter_{}.ckpt".format(e+1, i))
        print("epoch {} , avg_loss {}".format(e, loss/len(imgs)))
        print("\\\\\\\\\\\\\\\\\\\\\\\\")

In [ ]:
train(0, 0, 500)

iter 0 loss: 0.805573582649231
iter 500 loss: 0.20626862347126007
iter 1000 loss: 0.18054518103599548
iter 1500 loss: 0.26953405141830444


In [24]:
saver.save(sess, save_path="checkpoints/epoch_{}_iter_{}.ckpt".format(10, 10))

'checkpoints/epoch_10_iter_10.ckpt'

In [30]:
def plot_box(bbox, imgName):
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from PIL import Image
    import numpy as np
    w, h = 100, 100
    im = np.array(Image.open(imgName).resize((100, 100), Image.ANTIALIAS), dtype=np.uint8)
    fig,ax = plt.subplots(1)
    ax.imshow(im)
    for box in bbox:
        _, X, Y, W, H = box
        rect = patches.Rectangle((X * w ,Y * h), W * w, H * h ,linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
        
    plt.show()